In [ ]:
pip install nltk

In [ ]:
pip install jsonlines

In [ ]:
pip install transformers sentencepiece

In [ ]:
pip install torch

In [ ]:
pip install evaluate

In [ ]:
!pip install accelerate -U

In [ ]:
!pip install scikit-learn

In [122]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import jsonlines
import json
nltk.download('punkt')
from transformers import TrainingArguments
import numpy as np
from transformers import DataCollatorWithPadding
from transformers import Trainer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import Dataset
import pandas as pd

[nltk_data] Downloading package punkt to /home/ebakhtina/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [123]:
path = "tqdm-data.jsonl"
warnings = []
with open(path, 'rb') as f:
    for item in jsonlines.Reader(f):
        message = item.get("message")
        if message is not None:
            warnings.append(message)

In [124]:
sentences = []
sentences.append('message')
for w in warnings:
    tmp = sent_tokenize(w)
    for elem in tmp:
        sentences.append(elem)

In [28]:
from torch.utils import data
class CustomTextDataset(data.Dataset):
    def __init__(self, warnings, numbers, transform=None, target_transform=None):
        
        self.comments = warnings
        
        self.labels = numbers
        
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.comments)

    def __getitem__(self, idx):
        if idx < 0 or idx >= len(self.comments):
            raise IndexError("Index out of range")
        text = self.comments[idx]
        label = self.labels[idx]
        return text, label

In [99]:
from datasets import load_dataset
dset = load_dataset("json", data_files="tqdm-data.jsonl", split="train")

id = []
for i in range(len(dset)):
    id.append(i)
dset = dset.add_column("id", id)

from random import randint
numbers = []
for i in range(len(dset)):
    numbers.append(randint(0, 1))
dset = dset.add_column("labels", numbers)

def tokenize_function(examples):
	return tokenizer(examples['message'], padding='max_length', truncation=True)

tokenized_dset_of_comments = dset.map(tokenize_function)

print(tokenized_dset)

Map:   0%|          | 0/87 [00:00<?, ? examples/s]

Dataset({
    features: ['message', 'id', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 87
})


In [100]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True)
training_args = TrainingArguments(output_dir="test_trainer", learning_rate=6e-6, per_device_train_batch_size=4, num_train_epochs=3)
from accelerate import Accelerator
accelerator = Accelerator()
model, tokenizer = accelerator.prepare(model, tokenizer)
# dset = dset.rename_column("labels", "label")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [101]:
import evaluate
metric = evaluate.load("accuracy")

In [102]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred.predictions, eval_pred.label_ids
    predictions = np.argmax(logits, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [103]:
trainer = Trainer(
    model=model,
    tokenizer = tokenizer, 
    args=training_args,
    train_dataset=tokenized_dset_of_comments,
    eval_dataset=tokenized_dset_of_comments,
    compute_metrics=compute_metrics,
)

In [104]:
trainer.train()
result = trainer.evaluate()

Step,Training Loss


In [119]:
numbers_for_sentences = []
for i in range(len(sentences)):
    numbers_for_sentences.append(randint(0,1))
print(numbers_for_sentences)

[0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1]


In [120]:
prep_of_sent = []
for elem in sentences:
    for n in numbers_for_sentences:
        d = {elem: n}
        prep_of_sent.append(d)
        break
dataset_of_sentences = Dataset.from_list(prep_of_sent)
dataset_of_sentences = dataset_of_sentences.add_column("labels", numbers_for_sentences)

In [121]:
dataset_of_sentences

Dataset({
    features: ['message', 'labels'],
    num_rows: 170
})